In [ ]:
import pandas as pd 

from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.naive_bayes import MultinomialNB

import sys
sys.path.append('../')
import cleaner as dc

In [7]:
import pandas as pd

df_clean = pd.read_csv('../clean_dataset.csv')
df_clean.shape

(83796, 2)

In [8]:
# words and sequences that cab be used to filter our ads using scarcity
negative_emotion_words = ['suffering', 'suffered','nightmare','forced','abuse','abused','burden','getting worse','horror','violence',
                    'catastrophe','affraid','cruelty','deadly','terror', 'danger', 'dangerous',  'regret', 'regretted', 'rejection', 
                    'reject','rejected', 'risk', 'risked', 'sabotage', 'sabotaged', 'sabotaging', 'scary', 'scared','suffer', 'threat',
                    'threatened', 'threatening', 'tragic', 'tragically', 'tragical', 'trapped' , 'trap','fail', 'fails', 'failed', 'failing', 
                    'victim', 'fooled','fooling', 'helpless', 'hurting', 'hurt', 'scared','mistake' , 'mistaken', 'neglect','neglected', 'neglecting', 
                    'pitfalls', 'powerless', 'vulnerable', 'waste', 'wasted','wasting', 'worry', 'worried', 'worrying', 'disappointed', 'dissapoint', 
                    'dissapointing','frustrated', 'frsutrating', 'greedy',  'greed','irritated',  'maddening', 'mad', 'ruthless',  'misleading', 
                    'frustration','infuriating','infuriated', 'pointless', 'reclaim', 'reclaimed', 'sick', 'tired', 'tiring', 'swindled', 'swindle',
                    'trigger', 'triggered', 'unacceptable', 'violate', 'violated', 'uncomfortable', 'victims', 'risking', 'critcial', 'critically', 
                    'concerning', 'concerned', 'problamatic', 'drastic', 'drastiaclly', 'hardship', 'hardships', 'rigging', 'rape', 'raped', 
                    'assault', 'assaulted', 'assaulting', 'insult', 'insulting', 'murder', 'weapon',  'rigged', 'desperate', 'desperated', 
                    'danger', 'regret',  'risk', 'sabotage', 'scary','threat','tragic', 'victim',  'helpless', 'hurting','vulnerable', 
                    'frustrated', 'greedy','irritated',  'violate' ]
                    
# words and sequences that cab be used to filter our ads using scarcity
positive_emotion_words = ['delighted', 'ecstatic', 'freedom', 'relaxed', 'assured', 'happy', 'healthy',
                 'fulfilled', 'genuine', 'authentic', 'secure', 'stable', 'honest', 'truthful','supportive', 'excellent',
                'blissful', 'joyous', 'delighted', 'overjoyed', 'gleeful', 'thankful', 'festive', 'ecstatic', 'satisfied', 
                'cheerful','sunny', 'elated', 'jubilant', 'jovial', 'fun-loving', 'lighthearted', 'glorious', 'innocent', 
                'child-like', 'gratified', 'euphoric', 'playful', 'courageous', 'energetic', 'liberated', 'optimistic', 
                'animated', 'spirited','thrilled', 'wonderful', 'funny', 'exhilarated','youthful', 
                'tickled', 'creative', 'constructive', 'comfortable', 'pleased', 'encouraged', 'surprised', 
                'content', 'serene', 'bright', 'blessed', 'vibrant', 'glowing','relaxing','pleasing',
                'fulfilling','good quality','compassion','glad','appreciative', 'joyful', 'pleased', 
                'confident', 'cheery', 'polite', 'chilly','glory','brave', 'heroic', 'enthusiastic', 
                'passionate','optimistic','delighted', 'excited', 'amazing','amusing','innovative', 'artistic',
                'authentic' 'supportive', 'excellent','blissful', 'joyous', 'delighted', 'overjoyed', 'gleeful', 'thankful']

def contains_word(s, w):
    if w in s.lower() :
      return True
    else: 
      return False

In [10]:
# Sentimet analysis model 
df = pd.read_csv('../sentiment_dataset.csv')
print(df.shape)

# clean data 
df['review'] = df['review'].apply(lambda x: x.lower())
df['review'] = df['review'].apply(dc.remove_stopwords)
df['review'] = df['review'].str.replace('[^\w\s]','', regex=True)
df['review'] = df['review'].str.replace('\s+', ' ',  regex=True)
df['review'] = df['review'].apply(dc.denoise_text)
df['review'] = df['review'].apply(dc.remove_special_characters)
df['review'] = df['review'].apply(dc.simple_stemmer)

# split training and testing dataset 
#train dataset
norm_train=df.review[:40000]
train_sentiments=df.sentiment[:40000]

#test dataset
norm_test=df.review[40000:]
test_sentiments=df.sentiment[40000:]

# vectorize training and testing 
#Count vectorizer for bag of words
cv = CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
cv_train_reviews = cv.fit_transform(norm_train)
cv_test_reviews=cv.transform(norm_test)


# train naive bayes model for sentiment detection
mnb=MultinomialNB()
mnb_bow=mnb.fit(cv_train_reviews,train_sentiments)

(50000, 2)


In [ ]:
# use the trained model to predict sentiments in the negative emotional persuasion ads 
negative_emotion_true , negative_emotion_true_new =  set() , set() 
df_nemotion = pd.DataFrame() 

for sentence in df_clean['ads']:                                   
  for word in  negative_emotion_words:                                        
      if contains_word(sentence, word):
        negative_emotion_true.add(sentence)

# take ones that do not contain positive words
for sentence in negative_emotion_true:                                   
  for word in  positive_emotion_words:                                        
      if contains_word(sentence, word):
        negative_emotion_true_new.add(sentence)
negative_emotion_true = negative_emotion_true_new

df_negative_emotion_true = pd.DataFrame(list(negative_emotion_true), columns= ['ads'])
df_negative_emotion_true['stemmed'] = df_negative_emotion_true['ads'].apply(dc.simple_stemmer) # stemming the text 
df_negative_emotion_true['sentiment'] = 0 # negative sentiment 
print(df_negative_emotion_true.shape)

# make predictions
norm_data = cv.transform(df_negative_emotion_true['stemmed'])
df_negative_emotion_true['predicted']= mnb.predict(norm_data)
df_nemotion = df_negative_emotion_true.loc[df_negative_emotion_true['predicted'] == 'negative']
df_nemotion.drop(columns  = ['predicted', 'stemmed', 'sentiment' ], inplace = True)
df_nemotion['emotion'] = 1

In [19]:
df_nemotion.shape

(1014, 2)

In [ ]:
# use the trained model to predict sentiments in the positive emotional persuasion ads 
positive_emotion_true , positive_emotion_true_new =  set() , set() 
df_pemotion = pd.DataFrame() 

for sentence in df_clean['ads']:                                   
  for word in  positive_emotion_words:                                        
      if contains_word(sentence, word):
        positive_emotion_true.add(sentence)

# take ones that do not contain positive words
for sentence in positive_emotion_true:                                   
  for word in  positive_emotion_words:                                        
      if contains_word(sentence, word):
        positive_emotion_true_new.add(sentence)
positive_emotion_true = positive_emotion_true_new

df_positive_emotion_true = pd.DataFrame(list(positive_emotion_true), columns= ['ads'])
df_positive_emotion_true['stemmed'] = df_positive_emotion_true['ads'].apply(dc.simple_stemmer) # stemming the text 
df_positive_emotion_true['sentiment'] = 0 # positive sentiment 
print(df_positive_emotion_true.shape)

# make predictions
norm_data = cv.transform(df_positive_emotion_true['stemmed'])
df_positive_emotion_true['predicted']= mnb.predict(norm_data)
df_pemotion = df_positive_emotion_true.loc[df_positive_emotion_true['predicted'] == 'positive']
df_pemotion.drop(columns  = ['predicted', 'stemmed', 'sentiment' ], inplace = True)
df_pemotion['emotion'] = 1

In [21]:
df_pemotion.shape

(3079, 2)

In [25]:
df_emotional_appeal = pd.concat([df_nemotion, df_pemotion.sample(n = 1200)]).reset_index(drop = True)
df_emotional_appeal.shape

(2214, 2)